In [77]:
import pandas as pd
import requests
import mysql.connector

In [87]:
API_KEY = ''
CHANNEL_ID = 'UC2UXDak6o7rBm23k3Vv5dww'
PAGE_TOKEN = ''

searchUrl = 'https://www.googleapis.com/youtube/v3/search?key=' + API_KEY + '&part=snippet' + '&type=video' + '&channelId=UCG5qGWdu8nIRZqJ_GgDwQ-w' + '&order=date' + '&maxResults=50' + '&pageToken='
videosUrl = 'https://www.googleapis.com/youtube/v3/videos?key=' + API_KEY + '&part=snippet,contentDetails,statistics' + '&id='

df = pd.DataFrame(columns=['videoId', 'publishedAt', 'title', 'tags', 'duration', 'viewCount', 'likeCount', 'commentCount'])

In [88]:
while True:
    try:
        searchResponse = requests.get(searchUrl + PAGE_TOKEN).json()
        PAGE_TOKEN = searchResponse['nextPageToken']
        print(len(df.index))

        for video in searchResponse['items']:
            videoId = video['id']['videoId']
            videoResponse = requests.get(videosUrl + videoId).json()
            
            publishedAt = videoResponse['items'][0]['snippet']['publishedAt']
            title = videoResponse['items'][0]['snippet']['title']
            tags = videoResponse['items'][0]['snippet']['tags']

            duration = videoResponse['items'][0]['contentDetails']['duration']

            viewCount = videoResponse['items'][0]['statistics']['viewCount']
            likeCount = videoResponse['items'][0]['statistics']['likeCount']
            commentCount = videoResponse['items'][0]['statistics']['commentCount']

            df = df.append({'videoId' : videoId, 
                            'publishedAt' : publishedAt, 
                            'title' : title, 
                            'tags' : tags,
                            'duration' : duration,
                            'viewCount' : viewCount,
                            'likeCount' : likeCount,
                            'commentCount' : commentCount}, ignore_index=True)
    except:
        print(len(df.index))
        break

0
50
100
150
200
250
300
350
400
450
500


In [89]:
df

,videoId,publishedAt,title,tags,duration,viewCount,likeCount,commentCount
0,twmLtygXxCk,2023-06-15T15:01:21Z,Footballers with GENIUS vision! | Best Premier...,"[assists, assist, football assists, soccer ass...",PT19M32S,75390,2043,163
1,2_8cReiBhbY,2023-06-15T13:00:01Z,When you try to guess the answer early... 🤦‍♂️...,"[premier league, premier league highlights, pr...",PT15S,62508,2482,22
2,EvROzQeMee8,2023-06-15T09:32:25Z,Mohamed Salah doesn’t miss! #shorts,"[Mohamed Salah, Mo Salah, Salah, Liverpool, Li...",PT25S,503736,25031,128
3,4b4wUoVqD4o,2023-06-14T16:30:00Z,Do you know what happens next? 🤔 #shorts,"[premier league, premier league highlights, pr...",PT19S,414455,13039,61
4,Vrrt7a_1oes,2023-06-14T15:32:38Z,IMPOSSIBLE GOALKEEPER SAVES | Premier League |...,"[impossible goalkeeper saves, impossible saves...",PT14M11S,84491,1963,134
...,...,...,...,...,...,...,...,...
495,jWrSR-5Mob4,2021-07-08T15:11:49Z,The MOST HAT-TRICKS for every Premier League c...,"[Premier League, EPL, PL, Record Breaking, Hat...",PT26M25S,579212,6113,329
496,KpePssz9Uw0,2021-07-01T15:01:51Z,RECORD-BREAKING Clean sheets! Great Premier Le...,"[Premier League, EPL, PL, Record Breaking, Sac...",PT14M49S,451404,5948,289
497,1hRc7y7UIWI,2021-06-30T15:10:46Z,FIRST & LAST Premier League goals against the ...,"[#PremierLeague, #FirstGoal, #LastGoal, #Goals...",PT8M33S,267672,4537,160
498,Ot1O97maPps,2021-06-14T10:35:17Z,Premier League Stars at Euro 2020 | Group C | ...,"[Euro 2020, European Championship, Premier Lea...",PT4M31S,31102,793,107


In [90]:
mydb = mysql.connector.connect(
  host='localhost',
  user='root',
  password='Justocker70',
  database ='dfdb'
)
print(mydb)
mycursor = mydb.cursor()

In [41]:
#mycursor.execute('CREATE DATABASE dfdb')

#mycursor.execute('SHOW DATABASES')
#for db in mycursor:
#    print(db)

#mycursor.execute("""
#CREATE TABLE IF NOT EXISTS original (
#  video_id VARCHAR(255) PRIMARY KEY,
#  upload_date TEXT,
#  video_title TEXT,
#  video_tags TEXT,
#  video_duration TEXT,
#  video_view INTEGER,
#  video_like INTEGER,
#  video_comment INTEGER
#)""")

In [91]:
checkFormula = 'SELECT video_id FROM original WHERE video_id=%s'
updateForumla = 'UPDATE original SET video_view = %s, video_like = %s, video_comment = %s WHERE video_id = %s;'
insertForumla = 'INSERT INTO original (video_id, upload_date, video_title, video_tags, video_duration, video_view, video_like, video_comment) VALUES (%s, %s, %s, %s, %s, %s, %s, %s);'

for i, row in df.iterrows():
    mycursor.execute(checkFormula, (row['videoId'], ))
    if mycursor.fetchone() is not None: #check if it exists
        #update
        rowData = (
            row['viewCount'],
            row['likeCount'],
            row['commentCount'],
            row['videoId']
        )
        mycursor.execute(updateForumla, rowData)
    else:
        #insert
        rowData = (
            row['videoId'],
            row['publishedAt'],
            row['title'],
            ', '.join(row['tags']),
            row['duration'],
            row['viewCount'],
            row['likeCount'],
            row['commentCount']
        )
        mycursor.execute(insertForumla, rowData)

mydb.commit()